In [1]:
import pandas as pd
import numpy as np
import tqdm
import os

import logging
logging.basicConfig(format='%(asctime)s [%(levelname)s] %(message)s',
                    level=logging.INFO)

import nltk
from nltk import FreqDist

from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
import lxml
import re

In [2]:
root_dir = r"C:\Users\wb570559\github\india_air_pollution\data\processed_data\\"

In [3]:
df_yearly = pd.read_csv(root_dir+"yearly_air.csv").drop(["Unnamed: 0"], axis=1)

<ipython-input-3-f646ca6aea6e>:1: DtypeWarning: Columns (21,35,38,50,55,61,63,65,66,67,68,69,70,71,72,73,74,75,76,77,90,92,95,101,103,105,106,109,110,111,112,113,114,115,116,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,142,143,144,150,151,152,153,154,155,158,161,198,199,200,231) have mixed types. Specify dtype option on import or set low_memory=False.
  df_yearly = pd.read_csv(root_dir+"yearly_air.csv").drop(["Unnamed: 0"], axis=1)


In [4]:
df_cases = pd.read_csv(root_dir+"case_data_final_with_additional_variables.csv")

In [5]:
df_cases = df_cases[["Kanoon_ID", "length_of_case"]]

In [6]:
df_cases

,Kanoon_ID,length_of_case
0,1320.0,2548
1,1569.0,165
2,7577.0,12656
3,11626.0,3730
4,14406.0,4126
...,...,...
1905,198654408.0,335
1906,198796622.0,8348
1907,199228395.0,1147
1908,199426007.0,137


In [7]:
df_yearly = pd.merge(df_yearly, df_cases, left_on = "kanoon_id", right_on = "Kanoon_ID", how = "left")

In [8]:
df_yearly = df_yearly.drop(["Kanoon_ID_y"], axis=1)

In [9]:
df_yearly.head()

,state_x,district_x,year,month,bc,du,oc14,oc16,oc18,pm14,...,d2v_vec24,d2v_vec25,kanoon_id,judges,matched_judges,pollution_district,pollution_state,district_present,delhi_dummy,length_of_case
0,andaman and nicobar,andaman islands,1981.0,6.5,0.125165,1.710763,1.204337,1.376386,1.548434,11.200506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
1,andaman and nicobar,andaman islands,1982.0,6.5,0.161135,1.191211,1.888157,2.157893,2.427630,10.655111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
2,andaman and nicobar,andaman islands,1983.0,6.5,0.245145,1.024064,3.006749,3.436285,3.865820,11.420768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
3,andaman and nicobar,andaman islands,1984.0,6.5,0.137811,1.890178,1.414911,1.617042,1.819172,11.424865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
4,andaman and nicobar,andaman islands,1985.0,6.5,0.119531,1.885741,1.087987,1.243414,1.398841,11.346846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN


In [10]:
important_variables = ['impact_coded',
 'govt_respondent',
 'govt_petitioner',
 'is_appeal',
 'is_constitutional',
 'mean_coded_vals',
 'most_freq_coded_vals',
 'delivery_year',
 'gender_binary',
 'lsa_vec1',
 'lsa_vec2',
 'lsa_vec3',
 'lsa_vec4',
 'lsa_vec5',
 'lsa_vec6',
 'lsa_vec7',
 'lsa_vec8',
 'lsa_vec9',
 'lsa_vec10',
 'lsa_vec11',
 'lsa_vec12',
 'lsa_vec13',
 'lsa_vec14',
 'lsa_vec15',
 'lsa_vec16',
 'lsa_vec17',
 'lsa_vec18',
 'lsa_vec19',
 'lsa_vec20',
 'lsa_vec21',
 'lsa_vec22',
 'lsa_vec23',
 'lsa_vec24',
 'lsa_vec25',
 'd2v_vec1',
 'd2v_vec2',
 'd2v_vec3',
 'd2v_vec4',
 'd2v_vec5',
 'd2v_vec6',
 'd2v_vec7',
 'd2v_vec8',
 'd2v_vec9',
 'd2v_vec10',
 'd2v_vec11',
 'd2v_vec12',
 'd2v_vec13',
 'd2v_vec14',
 'd2v_vec15',
 'd2v_vec16',
 'd2v_vec17',
 'd2v_vec18',
 'd2v_vec19',
 'd2v_vec20',
 'd2v_vec21',
 'd2v_vec22',
 'd2v_vec23',
 'd2v_vec24',
 'd2v_vec25']

In [11]:
for var in important_variables:
    
    df_yearly[str(var)+"_len_heur"] = df_yearly[var]/df_yearly["length_of_case"]
    

In [12]:
df_yearly.describe()

,year,month,bc,du,oc14,oc16,oc18,pm14,pm16,pm18,...,d2v_vec16_len_heur,d2v_vec17_len_heur,d2v_vec18_len_heur,d2v_vec19_len_heur,d2v_vec20_len_heur,d2v_vec21_len_heur,d2v_vec22_len_heur,d2v_vec23_len_heur,d2v_vec24_len_heur,d2v_vec25_len_heur
count,54012.000000,54012.0,54012.000000,54012.000000,54012.000000,54012.000000,54012.000000,54012.000000,54012.000000,54012.000000,...,9976.000000,9976.000000,9976.000000,9976.000000,9976.000000,9976.000000,9976.000000,9976.000000,9976.000000,9976.000000
mean,2001.908057,6.5,1.410315,11.670257,9.594849,10.966981,12.337518,31.438925,32.807479,34.178809,...,-0.000846,-0.000984,0.000585,-0.000451,0.000419,0.000297,-0.000480,-0.000904,-0.000575,-0.000353
std,11.778729,0.0,0.935511,7.248926,5.710143,6.525762,7.342806,15.788455,16.480672,17.185281,...,0.001130,0.001342,0.001030,0.001008,0.000980,0.000977,0.000907,0.001375,0.001180,0.000855
min,1981.000000,6.5,0.044657,0.310470,0.305132,0.348722,0.392313,3.956820,4.143817,4.330814,...,-0.010510,-0.010097,-0.003091,-0.011340,-0.007982,-0.006331,-0.007849,-0.012098,-0.012252,-0.008334
25%,1992.000000,6.5,0.744427,6.134462,5.567675,6.362074,7.157821,20.192005,21.100445,21.951063,...,-0.001230,-0.001322,0.000066,-0.000443,0.000003,0.000027,-0.000608,-0.001091,-0.000775,-0.000428
50%,2003.000000,6.5,1.150830,10.077892,7.959024,9.095985,10.231025,26.864815,28.082307,29.316673,...,-0.000409,-0.000381,0.000286,-0.000087,0.000102,0.000120,-0.000198,-0.000397,-0.000168,-0.000073
75%,2012.000000,6.5,1.731329,15.537946,12.090580,13.829446,15.534521,39.606815,41.087135,42.711357,...,-0.000122,-0.000149,0.000703,-0.000003,0.000655,0.000511,-0.000043,-0.000114,-0.000045,-0.000009
max,2020.000000,6.5,4.544521,66.794050,36.178950,41.347374,46.515797,88.291275,89.374850,90.458420,...,0.001091,0.001814,0.010437,0.001779,0.008142,0.011632,0.004221,0.001323,0.003481,0.006141


In [13]:
df_yearly.to_csv(root_dir+"yearly_air_len_heuristic.csv", index=False)